In [1]:
import keras
import numpy as np

path = keras.utils.get_file(
        'nietzsche.txt',
        origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus len:', len(text))

Using TensorFlow backend.


606208/600901 [==============================] - 1s 1us/step
Corpus len: 600893


In [3]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Nr of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Nr of sequences: 200278
Unique characters: 57
Vectorization...


In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen -1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)
        sys.stdout.write(generated_text)
        
    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.
            
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        
        generated_text += next_char
        generated_text = generated_text[1:]
        
        sys.stdout.write(next_char)

epoch 1
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
200278/200278 [==============================] - 196s 978us/step - loss: 2.0044
--- Generating with seed: "e heart, which teaches the clumsy and too hasty hand to hesi"
----- temperature: 0.2
e heart, which teaches the clumsy and too hasty hand to hesi----- temperature: 0.5
e heart, which teaches the clumsy and too hasty hand to hesi----- temperature: 1.0
e heart, which teaches the clumsy and too hasty hand to hesi----- temperature: 1.2
e heart, which teaches the clumsy and too hasty hand to hesionely, hle fording a pose is will at fun wfill"
labinefual mur-
the: umbaticilops, objest indirilti que
look. that munrlinerouses unfor-creeftical, stmay:

 "preboster wreas ymmearsent of werates, wis, at peniousntion ta federicvabalty, and
recowutuless, imtertyely-purent at this promicateseught
and dive s2opect to
vering,", funcess as
subefural
fordion well abiepoin of which drcement, swerises,
pepoch 2
Epoch 1/1
200278/200278 [

KeyboardInterrupt: 